In [2]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

In [3]:
import psutil
psutil.virtual_memory()

svmem(total=59098333184, available=54803959808, percent=7.3, used=3605594112, free=47947501568, active=5260722176, inactive=4989837312, buffers=152236032, cached=7393001472, shared=9150464, slab=517611520)

In [4]:
df = pd.read_csv('parsed/corpus/2015.csv', index_col=0, na_filter=False)
docs = df.text.to_list()
with open('polish_stopwords.txt') as f:
    stop_words = [x.strip() for x in f]

vectorizer_model = CountVectorizer(stop_words=stop_words, ngram_range=(1, 3))

topic_model = BERTopic(embedding_model='xlm-r-distilroberta-base-paraphrase-v1', vectorizer_model=vectorizer_model, low_memory=True, verbose=True)

In [5]:
topics, _ = topic_model.fit_transform(docs)

Batches:   0%|          | 0/1309 [00:00<?, ?it/s]

2021-03-24 18:42:03,080 - BERTopic - Transformed documents to Embeddings
2021-03-24 18:42:52,574 - BERTopic - Reduced dimensionality with UMAP
2021-03-24 18:42:56,170 - BERTopic - Clustered UMAP embeddings with HDBSCAN


In [6]:
topic_model.save("model/2015-default", save_embedding_model=False)

In [8]:
topic_model.get_topic_freq().head()

,Topic,Count
0,-1,26517
1,161,1159
2,174,915
3,80,731
4,94,605


In [10]:
plot_topics = topic_model.visualize_topics()
plot_topics.write_html('plots/topics.html')
plot_topics

In [12]:
plot_topics

In [13]:
topics_over_time = topic_model.topics_over_time(df.text.to_list(), topics, df.date.to_list(), nr_bins=48)

46it [8:30:36, 666.01s/it]


In [14]:
plot_topics_over_time = topic_model.visualize_topics_over_time(topics_over_time, top_n=20)
plot_topics_over_time.write_html('plots/topics_over_time.html')
plot_topics_over_time

In [7]:
with open('model/topics.txt','w') as f:
    print(topics, file=f)

In [19]:
topic_model.find_topics('500 plus')

([170, 174, 138, 109, 224],
 [0.5323614429278389,
  0.5197511578332342,
  0.4896854203703309,
  0.48385468133325144,
  0.4775853005028442])

In [20]:
topic_model.get_topic(170)

[('wzrost', 0.007264657831947555),
 ('budżetu', 0.0051418359207242685),
 ('dochody podatkowe', 0.004786248197008863),
 ('dochodów', 0.004259583373127755),
 ('gospodarczego', 0.004188600107725766),
 ('poseł janusz cichoń', 0.003866289602085705),
 ('budżet', 0.0038628513137999167),
 ('gospodarki', 0.003737407823693195),
 ('polska gospodarka', 0.003674391478080321),
 ('przyrost', 0.0036611793147486625)]

In [15]:
topics_over_time.to_csv('model/topics_over_time.csv')

In [24]:
plot_topics.update_layout(width=1200, height=1200).write_html('plots/topics_cdn.html', include_plotlyjs='cdn')
plot_topics_over_time.update_layout(width=1920, height=700).write_html('plots/topics_over_time_cdn.html', include_plotlyjs='cdn')